In [ ]:
import re
import nltk
import string
import numpy as np
from nltk.cluster.util import cosine_distance
import networkx as nx

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [96]:
import spacy
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 84.4MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp37-none-any.whl size=21186283 sha256=496ee1656186d930235f57af197113dae623528d078c99a1fbcca3f75ac066fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-_yl7myjt/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [97]:
pln = spacy.load('pt')
pln

In [101]:
def preprocess(texto):
  formatado = texto.lower()
  tokens = []

  for token in nltk.word_tokenize(formatado):
    tokens.append(token)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return formatado

In [89]:
original = """A inteligência artificial é a inteligência similar à humana máquinas. 
              Definem como o estudo de agente artificial com inteligência. 
              Ciência e engenharia de produzir máquinas com inteligência. 
              Resolver problemas e possuir inteligência. 
              Relacionada ao comportamento inteligente. 
              Construção de máquinas para raciocinar. 
              Aprender com os erros e acertos. 
              Inteligência artificial é raciocinar nas situações do cotidiano."""
original = re.sub(r'\s+', ' ', original)
original

'A inteligência artificial é a inteligência similar à humana máquinas. Definem como o estudo de agente artificial com inteligência. Ciência e engenharia de produzir máquinas com inteligência. Resolver problemas e possuir inteligência. Relacionada ao comportamento inteligente. Construção de máquinas para raciocinar. Aprender com os erros e acertos. Inteligência artificial é raciocinar nas situações do cotidiano.'

In [12]:
originais = [sentenca for sentenca in nltk.sent_tokenize(original)]
formatadas = [preprocess(original) for original in originais]

In [13]:
originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [14]:
formatadas

['inteligência artificial inteligência similar humana máquinas',
 'definem estudo agente artificial inteligência',
 'ciência engenharia produzir máquinas inteligência',
 'resolver problemas possuir inteligência',
 'relacionada comportamento inteligente',
 'construção máquinas raciocinar',
 'aprender erros acertos',
 'inteligência artificial raciocinar situações cotidiano']

In [90]:
def simi(s1, s2):
  p1 = [palavra for palavra in nltk.word_tokenize(s1)]
  p2 = [palavra for palavra in nltk.word_tokenize(s2)]

  all = list(set(p1 + p2))

  v1 = [0] * len(all)
  v2 = [0] * len(all)

  for p in p1:
    v1[all.index(p)] += 1
  
  for p in p2:
    v2[all.index(p)] += 1
  
  return 1 - cosine_distance(v1, v2)

In [46]:
simi(formatadas[0], formatadas[1])

0.4743416490252569

In [91]:
def matriz(sentencas):
  ms = np.zeros((len(sentencas), len(sentencas)))
  
  for i in range(len(sentencas)):
    for j in range(len(sentencas)):
      if i != j:
        ms[i][j] = simi(sentencas[i], sentencas[j])
  return ms

In [48]:
matriz(formatadas)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])

In [92]:
def sum(texto, qt):
  originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  formatadas = [preprocess(original) for original in originais]

  ms = matriz(formatadas)
  
  grafo = nx.from_numpy_array(ms)
  
  pgrk = nx.pagerank(grafo)
  
  notas = sorted(((pgrk[i], nota) for i, nota in enumerate(originais)), reverse=True)
  
  melhores = []

  for i in range(qt):
    melhores.append(notas[i][1])

  return originais, melhores, notas

In [65]:
originais, melhores, notas = sum(original, 3)

In [ ]:
originais

In [ ]:
melhores

In [ ]:
notas

In [93]:
def resumo(titulo, sentencas, melhores):
  from IPython.core.display import HTML
  texto = ''

  display(HTML(f'<h1>Resumo - {titulo}</h1>'))
  for i in sentencas:
    if i in melhores:
      texto += str(i).replace(i, f'<mark>{i}</mark>')
    else:
      texto += i
  display(HTML(f" {texto} "))

In [94]:
resumo('teste', originais, melhores)

In [73]:
!pip install goose3

     |████████████████████████████████| 92kB 4.2MB/s 


In [74]:
from goose3 import Goose
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [75]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [102]:
originais, melhores, notas = sum(artigo.cleaned_text, 5)

In [103]:
resumo('Teste', originais, melhores)

In [104]:
def prelem(texto):
  formatado = texto.lower()
  texto = re.sub(r' +', ' ', texto)

  doc = pln(texto)
  tokens = []

  for token in doc:
    tokens.append(token.lemma_)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return formatado

In [105]:
def sumlem(texto, qt):
  originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  formatadas = [prelem(original) for original in originais]

  ms = matriz(formatadas)
  
  grafo = nx.from_numpy_array(ms)
  
  pgrk = nx.pagerank(grafo)
  
  notas = sorted(((pgrk[i], nota) for i, nota in enumerate(originais)), reverse=True)
  
  melhores = []

  for i in range(qt):
    melhores.append(notas[i][1])

  return originais, melhores, notas

In [106]:
originais, melhores, notas = sumlem(artigo.cleaned_text, 5)

In [107]:
resumo('Teste', originais, melhores)